In [5]:
import numpy as np
import cv2 as cv

In [45]:
# OpenCV Utility Class for Mouse Handling
class BlemishRemoval:
    def __init__(self, windowname, dests, colors_func):
    #def __init__(self, windowname, dests):
        self.prev_pt = None
        self.windowname = windowname
        self.dests = dests
        self.colors_func = colors_func
        self.dirty = False
        self.show()
        cv.setMouseCallback(self.windowname, self.on_mouse)

    def show(self):
        cv.imshow(self.windowname, self.dests[0])
        #cv.imshow(self.windowname + ": mask", self.dests[1])

    # onMouse function for Mouse Handling
    def on_mouse(self, event, x, y, flags, param):
        pt = (x, y)
        if event == cv.EVENT_LBUTTONDOWN:
            self.prev_pt = pt
        elif event == cv.EVENT_LBUTTONUP:
            self.prev_pt = None

        if self.prev_pt and flags & cv.EVENT_FLAG_LBUTTON:
            cv.circle(self.dests[1], self.prev_pt, int(15), (255,255,255),-1, 
                    cv.LINE_AA)
            self.dirty = True
            self.prev_pt = pt
            cv.inpaint(src=self.dests[0],dst = self.dests[0], inpaintMask=self.dests[1], inpaintRadius=3, flags=cv.INPAINT_TELEA)
            self.show()

In [46]:
# Read image in color mode
filename = 'acne.jpg'
img = cv.imread(filename, cv.IMREAD_COLOR)

# If image is not read properly, return error
if img is None:
    print('Failed to load image file: {}'.format(filename))

In [47]:
# Create a copy of original image
img_mask = img.copy()
# Create a black copy of original image
# Acts as a mask
inpaintMask = np.zeros(img.shape[:2], np.uint8)
# Create sketch using OpenCV Utility Class: BlemishRemoval
blemish = BlemishRemoval('image', [img_mask, inpaintMask], lambda : ((255, 255, 255), 255))
print("")
while True:
    ch = cv.waitKey()
    if ch == 27:
        break
    if ch == ord('r'):
        img_mask[:] = img
        inpaintMask[:] = 0
        blemish.show()
cv.destroyAllWindows()